In [1]:
import torch
print(torch.__version__)

1.13.1


In [13]:
import findspark


In [14]:

findspark.init()

In [15]:
from transformers import pipeline

In [16]:
import re
import pandas as pd

In [17]:
the_df = pd.read_csv(r"C:\Users\yasha\Desktop\yashas\sentimentanalysis\pythonProject\Cirkus.csv")

In [18]:
the_df

,Unnamed: 0,comment
0,0,IN\nSkip navigation\nSign in\n10:24\nTop 10 Wo...
1,1,NaN
2,2,NaN
3,3,Very few actors have a year like Pooja Hegde h...
4,4,Didn't we see this coming tho honestly? The tr...
...,...,...
745,745,NaN
746,746,NaN
747,747,NaN
748,748,NaN


In [19]:
my_df = the_df.dropna()
mylist = my_df['comment'].to_list()
del (mylist[0])

corpus = []
for i in range(len(mylist)):
    review = re.sub('[^a-zA-Z]', ' ', mylist[i])
    review = review.lower() 
    corpus.append(review)
            

In [20]:
corpus

['very few actors have a year like pooja hegde had  she left her mark in every industry   radhe shyam  beast  acharya and cirkus ',
 'didn t we see this coming tho honestly  the trailer itself was    well self explanatory  but do feel bad for jammy  our strongest warrior who puts up through sh t like this  we are forever indebted jammy',
 'pooja hegde does not disappoint in choosing bad movies  ',
 'never thought a genuinely bad ranveer singh performance was possible  but this is honestly right up there with ranbir in besharam and varun in coolie no    will never not get irritating when directors let accomplished actors get away with trashy performances ',
 'this is highest milestone in  matlab kuch bhi  genre from rohit shetty  n farhad samji is the writer btw  respect ',
 'if the comedy film has double role it meant to be disaster ',
 'i think rohit shetty s downfall critically started after chennai express in       that was last full masala film i liked  after that dilwale simmba   

In [21]:
scraped_comments = corpus.copy()

In [22]:
cleaned_df = pd.DataFrame(corpus)

In [23]:
cleaned_df

,0
0,very few actors have a year like pooja hegde h...
1,didn t we see this coming tho honestly the tr...
2,pooja hegde does not disappoint in choosing ba...
3,never thought a genuinely bad ranveer singh pe...
4,this is highest milestone in matlab kuch bhi ...
...,...
735,i know that movie is not good but the way you ...
736,aur sehen nahi hota mujhse
737,london thanks for letting us know you are her...
738,why can t we all agree ranveer singh is the mo...


In [24]:
import pyspark

In [25]:
from pyspark.sql import SparkSession

In [26]:
from pyspark import SparkContext, SparkConf

In [27]:
conf = SparkConf().setAppName('f').setMaster("local")
sc = SparkContext(conf=conf)

In [28]:
sc

<SparkContext master=local appName=f>

## distdata is our rdd

In [29]:
distData = sc.parallelize(corpus)

In [30]:
distData

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [31]:
distData.count()

740

In [32]:
distData.first()


'very few actors have a year like pooja hegde had  she left her mark in every industry   radhe shyam  beast  acharya and cirkus '

In [33]:

commentslist = distData.collect()

In [34]:
len(commentslist)

740

## Calling our model

In [35]:
model = pipeline(model="yashas123/finetuning-sentiment-model")

def sentiment(sentence):
    return model(sentence)
    
        

In [36]:
sentiment("i love this movie")

[{'label': 'LABEL_1', 'score': 0.9987161159515381}]

In [37]:
#def f(a):
#    print(a)
sentiment_list = []
sentiment_list = [sentiment(e) for e in commentslist]

In [38]:
sentiment_list

[[{'label': 'LABEL_0', 'score': 0.8186331987380981}],
 [{'label': 'LABEL_0', 'score': 0.9954965114593506}],
 [{'label': 'LABEL_0', 'score': 0.8768783211708069}],
 [{'label': 'LABEL_0', 'score': 0.9976772665977478}],
 [{'label': 'LABEL_1', 'score': 0.996052086353302}],
 [{'label': 'LABEL_0', 'score': 0.9969436526298523}],
 [{'label': 'LABEL_0', 'score': 0.8910700082778931}],
 [{'label': 'LABEL_1', 'score': 0.9415805339813232}],
 [{'label': 'LABEL_1', 'score': 0.9767357707023621}],
 [{'label': 'LABEL_1', 'score': 0.9972802400588989}],
 [{'label': 'LABEL_0', 'score': 0.9973216652870178}],
 [{'label': 'LABEL_1', 'score': 0.9431871175765991}],
 [{'label': 'LABEL_0', 'score': 0.9979304075241089}],
 [{'label': 'LABEL_0', 'score': 0.9978482723236084}],
 [{'label': 'LABEL_0', 'score': 0.9939024448394775}],
 [{'label': 'LABEL_1', 'score': 0.9972792267799377}],
 [{'label': 'LABEL_0', 'score': 0.9944276809692383}],
 [{'label': 'LABEL_0', 'score': 0.9784365296363831}],
 [{'label': 'LABEL_1', 'score

In [39]:
len(sentiment_list)

740

In [40]:
sentiment_list[18]

[{'label': 'LABEL_1', 'score': 0.9984643459320068}]

In [41]:
cleaned_df.iloc[18]

0    i also feel ranveer is amazing in serious role...
Name: 18, dtype: object

# new rdd creation

In [42]:
sentirdd = sc.parallelize(sentiment_list)

In [43]:
sentirdd.first()

[{'label': 'LABEL_0', 'score': 0.8186331987380981}]

In [44]:
rdd_with_id = sentirdd.zipWithIndex().map(lambda x: [{'label': d['label'], 'score': d['score'], 'id': x[1]} for d in x[0]])




In [45]:
row_with_id_list = []
row_with_id_list = [e for e in rdd_with_id.collect()]

In [46]:
len(row_with_id_list)

740

In [47]:
row_with_id_list

[[{'label': 'LABEL_0', 'score': 0.8186331987380981, 'id': 0}],
 [{'label': 'LABEL_0', 'score': 0.9954965114593506, 'id': 1}],
 [{'label': 'LABEL_0', 'score': 0.8768783211708069, 'id': 2}],
 [{'label': 'LABEL_0', 'score': 0.9976772665977478, 'id': 3}],
 [{'label': 'LABEL_1', 'score': 0.996052086353302, 'id': 4}],
 [{'label': 'LABEL_0', 'score': 0.9969436526298523, 'id': 5}],
 [{'label': 'LABEL_0', 'score': 0.8910700082778931, 'id': 6}],
 [{'label': 'LABEL_1', 'score': 0.9415805339813232, 'id': 7}],
 [{'label': 'LABEL_1', 'score': 0.9767357707023621, 'id': 8}],
 [{'label': 'LABEL_1', 'score': 0.9972802400588989, 'id': 9}],
 [{'label': 'LABEL_0', 'score': 0.9973216652870178, 'id': 10}],
 [{'label': 'LABEL_1', 'score': 0.9431871175765991, 'id': 11}],
 [{'label': 'LABEL_0', 'score': 0.9979304075241089, 'id': 12}],
 [{'label': 'LABEL_0', 'score': 0.9978482723236084, 'id': 13}],
 [{'label': 'LABEL_0', 'score': 0.9939024448394775, 'id': 14}],
 [{'label': 'LABEL_1', 'score': 0.9972792267799377,

In [48]:
rdd_with_id.first()

[{'label': 'LABEL_0', 'score': 0.8186331987380981, 'id': 0}]

In [49]:
rdd_with_id.count()

740

In [50]:
cleaned_df.iloc[0]

0    very few actors have a year like pooja hegde h...
Name: 0, dtype: object

In [51]:

rounded_rdd = rdd_with_id.map(lambda x: [{'label': d['label'], 'score': round(d['score'] * 100, 1), 'id': d['id']} for d in x])



In [52]:
rounded_rdd.first()

[{'label': 'LABEL_0', 'score': 81.9, 'id': 0}]

In [53]:
rounded_rdd.count()

740

In [54]:
from pyspark.sql import SparkSession

# create SparkSession
spark = SparkSession.builder.appName("RDD_to_DataFrame").getOrCreate()




In [55]:
spark

In [56]:
from pyspark.sql import Row

# Create a Row object for each element in the RDD
row_rdd = rounded_rdd.map(lambda x: Row(label=x[0]['label'], score=x[0]['score'], id=x[0]['id']))

# Convert the Row RDD to a DataFrame
outputdf = spark.createDataFrame(row_rdd)

# Show the DataFrame
outputdf.show()


+-------+-----+---+
|  label|score| id|
+-------+-----+---+
|LABEL_0| 81.9|  0|
|LABEL_0| 99.5|  1|
|LABEL_0| 87.7|  2|
|LABEL_0| 99.8|  3|
|LABEL_1| 99.6|  4|
|LABEL_0| 99.7|  5|
|LABEL_0| 89.1|  6|
|LABEL_1| 94.2|  7|
|LABEL_1| 97.7|  8|
|LABEL_1| 99.7|  9|
|LABEL_0| 99.7| 10|
|LABEL_1| 94.3| 11|
|LABEL_0| 99.8| 12|
|LABEL_0| 99.8| 13|
|LABEL_0| 99.4| 14|
|LABEL_1| 99.7| 15|
|LABEL_0| 99.4| 16|
|LABEL_0| 97.8| 17|
|LABEL_1| 99.8| 18|
|LABEL_0| 95.4| 19|
+-------+-----+---+
only showing top 20 rows



In [57]:
len(scraped_comments)

740

In [58]:
outputdf

DataFrame[label: string, score: double, id: bigint]

In [59]:
type(outputdf)

pyspark.sql.dataframe.DataFrame

In [60]:
outputdf.head(6)

[Row(label='LABEL_0', score=81.9, id=0),
 Row(label='LABEL_0', score=99.5, id=1),
 Row(label='LABEL_0', score=87.7, id=2),
 Row(label='LABEL_0', score=99.8, id=3),
 Row(label='LABEL_1', score=99.6, id=4),
 Row(label='LABEL_0', score=99.7, id=5)]

In [61]:
#from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# create a SparkSession
#spark = SparkSession.builder.appName("corpus_dataframe").getOrCreate()

# create an RDD from the corpus list
thecorpus_rdd = spark.sparkContext.parallelize(scraped_comments)

# create a schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("comment", StringType(), True)
])



In [62]:



# Filter out empty sentences
thecorpus_rdd = thecorpus_rdd.filter(lambda x: x.strip() != "")



# Create DataFrame with id and comment columns
inputdf= spark.createDataFrame(thecorpus_rdd.zipWithIndex().map(lambda x: (x[1], x[0])), schema)


In [63]:
inputdf.show()

+---+--------------------+
| id|             comment|
+---+--------------------+
|  0|very few actors h...|
|  1|didn t we see thi...|
|  2|pooja hegde does ...|
|  3|never thought a g...|
|  4|this is highest m...|
|  5|if the comedy fil...|
|  6|i think rohit she...|
|  7|movies like this ...|
|  8|perhaps an unpopu...|
|  9|back in       roh...|
| 10|pooja hegde and j...|
| 11|pooja hegde seems...|
| 12|this is one of th...|
| 13|a reality check w...|
| 14|the trailer for t...|
| 15|i have new found ...|
| 16|even by rohit she...|
| 17|originally writte...|
| 18|i also feel ranve...|
| 19|i totally agree w...|
+---+--------------------+
only showing top 20 rows



In [64]:
outputdf.show()

+-------+-----+---+
|  label|score| id|
+-------+-----+---+
|LABEL_0| 81.9|  0|
|LABEL_0| 99.5|  1|
|LABEL_0| 87.7|  2|
|LABEL_0| 99.8|  3|
|LABEL_1| 99.6|  4|
|LABEL_0| 99.7|  5|
|LABEL_0| 89.1|  6|
|LABEL_1| 94.2|  7|
|LABEL_1| 97.7|  8|
|LABEL_1| 99.7|  9|
|LABEL_0| 99.7| 10|
|LABEL_1| 94.3| 11|
|LABEL_0| 99.8| 12|
|LABEL_0| 99.8| 13|
|LABEL_0| 99.4| 14|
|LABEL_1| 99.7| 15|
|LABEL_0| 99.4| 16|
|LABEL_0| 97.8| 17|
|LABEL_1| 99.8| 18|
|LABEL_0| 95.4| 19|
+-------+-----+---+
only showing top 20 rows



In [65]:

joined_df = inputdf.join(outputdf, on=['id'], how='inner')

In [66]:
joined_df.count()

739

In [67]:
joined_df.show()


+---+--------------------+-------+-----+
| id|             comment|  label|score|
+---+--------------------+-------+-----+
| 26|honestly  rohit s...|LABEL_0| 98.8|
| 29|at the time of tr...|LABEL_1| 96.6|
|474|expecting anythin...|LABEL_0| 99.8|
| 65|sorry you had to ...|LABEL_0| 98.6|
|191|the fact that i t...|LABEL_0| 99.7|
|418|is it only me or ...|LABEL_1| 60.3|
|541|just watch  govin...|LABEL_1| 98.0|
|558|review  masooda  ...|LABEL_0| 98.2|
|222|bro  please watch...|LABEL_1| 81.1|
|270|worst film of the...|LABEL_0| 99.7|
|293|jammy     i am sh...|LABEL_0| 99.8|
|730|this is one of th...|LABEL_1| 79.7|
|243|thank you bollywo...|LABEL_1| 80.2|
|278|sajid khan ki him...|LABEL_0| 97.9|
|367|since you didn t ...|LABEL_1| 98.7|
|442|sajid khan must b...|LABEL_1| 99.6|
|705|overconfident  an...|LABEL_1| 83.2|
|720|year humshakals a...|LABEL_0| 99.5|
| 19|i totally agree w...|LABEL_0| 95.4|
| 54|jammy you have my...|LABEL_1| 91.9|
+---+--------------------+-------+-----+
only showing top

In [68]:
sorted_joined_df = joined_df.orderBy('id')
sorted_joined_df.show()


+---+--------------------+-------+-----+
| id|             comment|  label|score|
+---+--------------------+-------+-----+
|  0|very few actors h...|LABEL_0| 81.9|
|  1|didn t we see thi...|LABEL_0| 99.5|
|  2|pooja hegde does ...|LABEL_0| 87.7|
|  3|never thought a g...|LABEL_0| 99.8|
|  4|this is highest m...|LABEL_1| 99.6|
|  5|if the comedy fil...|LABEL_0| 99.7|
|  6|i think rohit she...|LABEL_0| 89.1|
|  7|movies like this ...|LABEL_1| 94.2|
|  8|perhaps an unpopu...|LABEL_1| 97.7|
|  9|back in       roh...|LABEL_1| 99.7|
| 10|pooja hegde and j...|LABEL_0| 99.7|
| 11|pooja hegde seems...|LABEL_1| 94.3|
| 12|this is one of th...|LABEL_0| 99.8|
| 13|a reality check w...|LABEL_0| 99.8|
| 14|the trailer for t...|LABEL_0| 99.4|
| 15|i have new found ...|LABEL_1| 99.7|
| 16|even by rohit she...|LABEL_0| 99.4|
| 17|originally writte...|LABEL_0| 97.8|
| 18|i also feel ranve...|LABEL_1| 99.8|
| 19|i totally agree w...|LABEL_0| 95.4|
+---+--------------------+-------+-----+
only showing top

In [69]:
#sorted_joined_df.write.csv('Avataroutput321full.csv',header=True)

In [70]:
from pyspark.sql.functions import col

# Create dataframe with rows having label value"LABEL_1")
 as LABEL_1
label1_df = sorted_joined_df.filter(col("label") == 
# Create dataframe with rows having label value as LABEL_0
label0_df = sorted_joined_df.filter(col("label") == "LABEL_0")


In [71]:
#label1_df.write.csv('Avatar320label1.csv',header=True)

In [75]:
label1_df.show()

+---+--------------------+-------+-----+
| id|             comment|  label|score|
+---+--------------------+-------+-----+
|  4|this is highest m...|LABEL_1| 99.6|
|  7|movies like this ...|LABEL_1| 94.2|
|  8|perhaps an unpopu...|LABEL_1| 97.7|
|  9|back in       roh...|LABEL_1| 99.7|
| 11|pooja hegde seems...|LABEL_1| 94.3|
| 15|i have new found ...|LABEL_1| 99.7|
| 18|i also feel ranve...|LABEL_1| 99.8|
| 22|the fact that you...|LABEL_1| 91.5|
| 23|finally people ar...|LABEL_1| 99.7|
| 24|i recently came a...|LABEL_1| 95.5|
| 25|this should ve be...|LABEL_1| 99.8|
| 28|after nani s gorg...|LABEL_1| 99.7|
| 29|at the time of tr...|LABEL_1| 96.6|
| 30| roy  joy and the...|LABEL_1| 99.8|
| 31|the one thing i l...|LABEL_1| 99.8|
| 34|the real robbery ...|LABEL_1| 99.4|
| 35|i think sajid kha...|LABEL_1| 95.5|
| 37| bhrantibilas  in...|LABEL_1| 99.9|
| 38|the amount of pro...|LABEL_1| 99.5|
| 39|they often say ma...|LABEL_1| 99.8|
+---+--------------------+-------+-----+
only showing top

In [76]:
total_positive_comments_as_per_model = label1_df.count()

In [77]:
#label0_df.write.csv('Avatar320label0.csv',header=True)

In [78]:
label0_df.show()

+---+--------------------+-------+-----+
| id|             comment|  label|score|
+---+--------------------+-------+-----+
|  0|very few actors h...|LABEL_0| 81.9|
|  1|didn t we see thi...|LABEL_0| 99.5|
|  2|pooja hegde does ...|LABEL_0| 87.7|
|  3|never thought a g...|LABEL_0| 99.8|
|  5|if the comedy fil...|LABEL_0| 99.7|
|  6|i think rohit she...|LABEL_0| 89.1|
| 10|pooja hegde and j...|LABEL_0| 99.7|
| 12|this is one of th...|LABEL_0| 99.8|
| 13|a reality check w...|LABEL_0| 99.8|
| 14|the trailer for t...|LABEL_0| 99.4|
| 16|even by rohit she...|LABEL_0| 99.4|
| 17|originally writte...|LABEL_0| 97.8|
| 19|i totally agree w...|LABEL_0| 95.4|
| 20|in the current cl...|LABEL_0| 70.6|
| 21|only surviving th...|LABEL_0| 90.4|
| 26|honestly  rohit s...|LABEL_0| 98.8|
| 27|hey  no teletubbi...|LABEL_0| 96.6|
| 32|those days are go...|LABEL_0| 83.0|
| 33|ranveer has to se...|LABEL_0| 99.7|
| 36|jammy s first lin...|LABEL_0| 99.0|
+---+--------------------+-------+-----+
only showing top

In [79]:
total_negative_comments_as_per_model =label0_df.count()

In [80]:
overall_sentiment=max(total_negative_comments_as_per_model,total_positive_comments_as_per_model)

In [81]:
def final_verdict():
    verdict = ""
    if overall_sentiment == total_negative_comments_as_per_model:
        verdict = "negative"
    elif overall_sentiment == total_positive_comments_as_per_model:
        verdict = "positive"
    else:
        verdict = "neutral"
    return verdict
    
     

In [82]:
final_verdict()

'negative'

In [83]:
total_negative_comments_as_per_model

401

In [84]:
total_positive_comments_as_per_model

338

my django api data modelling

In [1]:
import pandas as pd

In [7]:
columns = ["movie","total_comments","total_positive_comments","total_negative_comments","final_verdict"]

In [8]:

finaldf = pd.DataFrame(columns=columns)


In [9]:
finaldf

,movie,total_comments,total_positive_comments,total_negative_comments,final_verdict


In [10]:
#finaldf.loc[0] = {'movie': 'Avatar','total_comments':321, 'total_positive_comments': 254, 'total_negative_comments': 67, 'final_verdict': 'positive'}


In [87]:
#finaldf.loc[1] = {'movie': 'Cirkus','total_comments':739, 'total_positive_comments': 338, 'total_negative_comments': 401, 'final_verdict': 'negative'}

In [97]:
finaldf.loc[2] = {'movie': 'Pathan','total_comments':80, 'total_positive_comments': 67, 'total_negative_comments': 13, 'final_verdict': 'positive'}

In [99]:
finaldf

,movie,total_comments,total_positive_comments,total_negative_comments,final_verdict
0,Avatar,321,254,67,positive
1,Cirkus,739,338,401,negative
2,Pathan,80,67,13,positive


In [100]:
finaldf.to_csv("results.csv")

# my model does not understand 1)sarcasm,2)referenses to other movies it has not seen 3)humour,4) some hindi words written in english like "matlab kuch bhi"  --- disadvantages

# my model properly understands 
# 1) context switch 2) importance and giving weightage to words 3) technically better than all the papers referenced from previous published papers

# observations
9 - it is negative indeed 
14 - is actually positive but its an emotional statement ,
20 - is also positive
22 - model is unsure it gives only 63.3 negative but it is positive
25 - is positive 
26 - it understood context switch and gave it negative
29-it is a negative indeed
32 - actually negative with commentators parcepective as he needed hrithick
37 - he wanted john but he did not get so negative which is ok
49 - hrithick wasnt there emotion was not understood
53 -actually negative
55 -neutral since supressing is used might be then model has predicted like that
58- emotion that is why it is only 88 percent sure
65-yes negative
66-emotion
69-yes negative
75-abe word is used maybe it is the reason


In [95]:
t= pd.read_csv(r"C:\Users\yasha\Downloads\archive\IMDB Dataset.csv")

In [96]:
t.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
